# Zomato Geospatial Data Analysis
Weekly challenge: Week 7b  
Date: 9/6/2022

## Import libraries

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [9]:
df = pd.read_csv(r'C:\Users\unpat\_Projects\Challenge_AprojectAweek_data_files\Week_07\zomato.csv')
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [10]:
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

There are missing values in the location column. I am doing a spacial analysis here so will delete all rows with missing location data.

In [12]:
df.dropna(axis = 0, how = 'any', subset = ['location'], inplace = True)
# https://www.w3resource.com/pandas/dataframe/dataframe-dropna.php

In [13]:
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7754
votes                              0
phone                           1187
location                           0
rest_type                        206
dish_liked                     28057
cuisines                          24
approx_cost(for two people)      325
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [17]:
len(df['location'].unique())

93

There are 93 unique locations in this dataset.

In [18]:
# Creating a blank dataframe that will be used for the unique locations
locations = pd.DataFrame()

In [20]:
# Create a new column in the blank dataframe and adding the unique locations
locations['Name'] = df['location'].unique()

In [22]:
locations.head()

,Name
0,Banashankari
1,Basavanagudi
2,Mysore Road
3,Jayanagar
4,Kumaraswamy Layout


In [26]:
# Installing 'geopy' library which will help us extract the latitude and longitude of each location 
#!pip install geopy

In [27]:
from geopy.geocoders import Nominatim

In [28]:
geolocator = Nominatim(user_agent = 'app')

In [30]:
# Finding latitude and longitude for each location
lat = []
lon = []

for location in locations['Name']:
    # converting location from string to some geographical format using geocode
    location = geolocator.geocode(location)
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)        

In [52]:
print('LAT:\n', lat, '\n')
print('LON:\n', lon)

LAT:
 [15.8876779, 12.9417261, 12.9467026, 27.64392675, 12.9081487, 12.9274413, 22.8359967, 12.9055682, 12.2655944, 12.8845448, 39.76880625, 12.95961755, 12.8545439, 45.95485055, 12.5560169, 12.9089453, 17.2510682, 12.848759900000001, 18.1475, 12.9552572, 12.920441, 12.9489339, 22.478459, 12.9348429, 12.9408685, 40.5758316, 12.9364846, 13.6285774, 12.9327778, 12.93103185, 53.5533682, 12.9845687, 41.527815, 12.9732913, 12.9275886, 12.996845, 13.0227204, 12.9755264, 12.9736132, 40.7652844, 40.7128637, 12.9778793, 13.0530002, 18.5322493, 12.9837879, 33.117255, 36.5348643, 51.1689493, 51.5182611, 12.988721250000001, 13.0358698, 12.9624669, 12.945245, 12.9678074, 13.1261747, 13.0027353, 12.9931876, 13.0093455, 12.9390255, 1.2847055, 12.957998, 13.0101286, 30.1915796, 33.5935063, 12.9804194, 12.9413238, 13.007516, 12.9243692, 12.9282918, 13.2923988, 12.7450586, 12.9882338, 13.0141618, 12.9767936, 13.2227, 13.0258087, 13.0221416, 13.0435554, 13.0784743, nan, 12.973936, 12.9846713, 13.0382184,

In [53]:
# Creating 2 columns for 'lat' and 'lon' and storing the lat & lon values
locations['lat'] = lat
locations['lon'] = lon

In [54]:
locations.head()

,Name,lat,lon
0,Banashankari,15.887678,75.704678
1,Basavanagudi,12.941726,77.575502
2,Mysore Road,12.946703,77.530070
3,Jayanagar,27.643927,83.052805
4,Kumaraswamy Layout,12.908149,77.555318


In [56]:
# Converting locations dataframe into a .csv file
locations.to_csv(r'C:\Users\unpat\_Projects\Challenge_AProjectAWeek\Week_07\zomato_locations.csv', index = False)

In [62]:
# Counting the # of restaurants per location and converting it to a dataframe
Rest_locations = df['location'].value_counts().reset_index()
Rest_locations

,index,location
0,BTM,5124
1,HSR,2523
2,Koramangala 5th Block,2504
3,JP Nagar,2235
4,Whitefield,2144
...,...,...
88,West Bangalore,6
89,Yelahanka,6
90,Jakkur,3
91,Rajarajeshwari Nagar,2


In [64]:
# Renaming columns
Rest_locations.columns = ['Name', 'count']
Rest_locations

,Name,count
0,BTM,5124
1,HSR,2523
2,Koramangala 5th Block,2504
3,JP Nagar,2235
4,Whitefield,2144
...,...,...
88,West Bangalore,6
89,Yelahanka,6
90,Jakkur,3
91,Rajarajeshwari Nagar,2


In [65]:
# Merging locations and Rest_locations dataframes on Name column; dropping na values
Restaurant_locations = Rest_locations.merge(locations, on = 'Name', how = 'left').dropna()
Restaurant_locations.head()

,Name,count,lat,lon
0,BTM,5124,45.954851,-112.496595
1,HSR,2523,18.147500,41.538889
2,Koramangala 5th Block,2504,12.934843,77.618977
3,JP Nagar,2235,12.265594,76.646540
4,Whitefield,2144,53.553368,-2.296902
